In [1]:
import scanpy as sc
import pandas as pd
import glob, os

# Read metadata as a flat DataFrame (we’ll merge on two columns)
meta = pd.read_csv("GSE212217_seurat_scRNAseq_metadata.txt", sep="\t")
print("Metadata:", meta.shape, "columns:", meta.columns.tolist())

# Find all your HDF5s
h5_files = sorted(glob.glob("GSM*_scRNA_filtered_feature_bc_matrix.h5"))
print("Found", len(h5_files), "files, e.g.:", h5_files[:2])

adatas = []

for f in h5_files:
    sample = os.path.basename(f).split("_")[0]
    print(f"→ loading {sample} from {f}")
    
    # load with Scanpy
    ad = sc.read_10x_h5(f)
    # join the metadata on BOTH barcode and orig.ident
    #    first—extract the sample‐ID that matches your meta.orig.ident
    ident = os.path.basename(f).split("_")[1]      # e.g. "PEM1C1"
    ad.obs["orig.ident"] = ident
    
    #    bring the obs index (the 10× barcode) into a column so we can merge on it
    ad.obs["cellbarcode"] = ad.obs.index.astype(str)
    
    #    reset_index so everything is in columns, merge, then restore the barcode index
    meta2 = meta.reset_index()  # so meta2 has a "cellbarcode" column + orig.ident
    meta2["cellbarcode"] = meta2["cellbarcode"].str.replace(r"(_\d+)+$", "", regex=True)
    
    obs = (
        ad.obs.reset_index(drop=True)
               .merge(meta2, on=["cellbarcode","orig.ident"], how="left")
               .set_index("cellbarcode")
    )


    mask = obs["nCount_RNA"].notna()

    obs = obs[mask].copy()
    ad = ad[mask.values, :]

    # rebuild the composite index
    composite = obs.index + "_" + obs["orig.ident"]
    obs.index = composite
    ad.obs = obs
    ad.obs_names = composite
    ad.obs["sample"] = ident
    
    #    put it back into your AnnData
    ad.obs = obs
    
    #  now add your sample label
    ad.obs["sample"] = sample
    ad.var_names_make_unique()
    print(f"   → {ad.n_obs} cells × {ad.n_vars} genes, now obs has {ad.obs.shape[1]} columns")
    adatas.append(ad)

/loc/scratch/25752876/ipykernel_17209/3256387212.py:6: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  meta = pd.read_csv("GSE212217_seurat_scRNAseq_metadata.txt", sep="\t")


Metadata: (315843, 14) columns: ['cellbarcode', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'propMt', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.9', 'seurat_clusters', 'timepoint', 'timepointBinary', 'patient', 'clinical', 'finalIdent']
Found 52 files, e.g.: ['GSM6514096_PEM1C1_scRNA_filtered_feature_bc_matrix.h5', 'GSM6514097_PEM1C3_scRNA_filtered_feature_bc_matrix.h5']
→ loading GSM6514096 from GSM6514096_PEM1C1_scRNA_filtered_feature_bc_matrix.h5


/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


   → 4831 cells × 36601 genes, now obs has 15 columns
→ loading GSM6514097 from GSM6514097_PEM1C3_scRNA_filtered_feature_bc_matrix.h5


/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


...

/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


   → 65 cells × 36601 genes, now obs has 15 columns
→ loading GSM6514147 from GSM6514147_PEM23C5_scRNA_filtered_feature_bc_matrix.h5


/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


   → 6244 cells × 36601 genes, now obs has 15 columns


/app/software/scanpy/1.9.3-foss-2022b/lib/python3.10/site-packages/anndata/_core/anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [2]:
adata_all = sc.concat(
    adatas,
    join="outer",
    label="sample",
    index_unique=None
)
print("Combined AnnData:", adata_all)

Combined AnnData: AnnData object with n_obs × n_vars = 259834 × 36601
    obs: 'orig.ident', 'index', 'nCount_RNA', 'nFeature_RNA', 'propMt', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.9', 'seurat_clusters', 'timepoint', 'timepointBinary', 'patient', 'clinical', 'finalIdent', 'sample'


In [3]:
print("Combined AnnData:", adata_all)
print("Combined obs columns:", adata_all.obs.columns.tolist())

Combined AnnData: AnnData object with n_obs × n_vars = 259834 × 36601
    obs: 'orig.ident', 'index', 'nCount_RNA', 'nFeature_RNA', 'propMt', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.9', 'seurat_clusters', 'timepoint', 'timepointBinary', 'patient', 'clinical', 'finalIdent', 'sample'
Combined obs columns: ['orig.ident', 'index', 'nCount_RNA', 'nFeature_RNA', 'propMt', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.9', 'seurat_clusters', 'timepoint', 'timepointBinary', 'patient', 'clinical', 'finalIdent', 'sample']


In [4]:
for col in adata_all.obs.columns:
    if adata_all.obs[col].dtype == object:
        adata_all.obs[col] = adata_all.obs[col].fillna("").astype(str)

In [5]:
#save out
adata_all.write_h5ad("GSE212217_all_samples_with_metadata.h5ad")
print("Wrote merged AnnData ➔ GSE212217_all_samples_with_metadata.h5ad")

Wrote merged AnnData ➔ GSE212217_all_samples_with_metadata.h5ad
